In this notebook I created a .csv file, which consists of 10 columns: labels for the type of disease,the image id, the path of the .dcm file in the file system, the coordinates and size of the bounding boxes.The entries in the table are repeated because there can be several bounding boxes for one picture. Coordinates of bounding boxes are calculated for the size of pictures 512х512. You can change the coordinates and sizes of the bounding boxes by making elementary proportions :)

In [ ]:
import pydicom
import glob
import pandas as pd
import ast
import os

In [ ]:
image = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
image['StudyInstanceUID'] = image['StudyInstanceUID'] + '_study'
image

In [ ]:
study = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
study = study.set_index(['id'])
study

In [ ]:
labels = study.loc[image['StudyInstanceUID'].to_numpy()].reset_index()
labels

In [ ]:
df = pd.merge(image.rename(columns={'id': 'image_id'}), labels, how='inner', left_on='StudyInstanceUID', right_on='id')
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df = df.drop(['StudyInstanceUID', 'label', 'id'], axis=1)
df['image_id'] = df['image_id'].apply(lambda x: x[:-6])
df

In [ ]:
paths = glob.glob('../input/siim-covid19-detection/train/*/*/*.dcm')

image_idx = [os.path.split(path)[1][:-4] for path in paths]
image_idx[:5]

In [ ]:
paths_df = pd.DataFrame({'image_id': image_idx, 'Path': paths})
paths_df

In [ ]:
data = df.merge(paths_df, left_on='image_id', right_on='image_id')
data

In [ ]:
def get_wh(path):
    ds = pydicom.dcmread(path)
    # .Rows = height .Columns = width
    return {'w': ds.Columns, 'h': ds.Rows}

In [ ]:
# this cell is executed for a long time
wh = pd.DataFrame(list(data['Path'].apply(get_wh)))
wh

In [ ]:
data[['w', 'h']] = wh
data

In [ ]:
data['boxes'] = data['boxes'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) else x)
data = data.explode('boxes').reset_index(drop=True)
data

In [ ]:
data.loc[data['boxes'].isna(), 'boxes'] = str(dict(x=0, y=0, width=0, height=0))
data

In [ ]:
def str_to_dict(s):
    if type(s) is str:
        s = ast.literal_eval(s)
    return s

In [ ]:
data['boxes'] = data['boxes'].apply(str_to_dict)

In [ ]:
xywh = pd.DataFrame(list(data['boxes']))
xywh

In [ ]:
data[['x', 'y', 'width', 'height']] = xywh
data

In [ ]:
data['x'] = data['x'] / data['w'] * 512
data['y'] = data['y'] / data['h'] * 512
data['width'] = data['width'] / data['w'] * 512
data['height'] = data['height'] / data['h'] * 512
data

In [ ]:
df_to_save = data.drop(['boxes', 'w', 'h'], axis=1).rename(columns={'width': 'w', 'height': 'h'})
df_to_save

In [ ]:
df_to_save.to_csv('train.csv', index=False)